In [66]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import scipy.optimize as spopt

In [2]:
fp = pd.read_csv("synthetic_data.csv")

In [26]:
fp

,Unnamed: 0,Bakery,Garniers,Main,Salads,Soups,Cafes,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,latitude,longitude,vegan,Specificity
0,0,5,0,95,0,0,Singing Tree Bar and Grille,4.442549,4.673061,5.170736,4.781108,6.233308,4.241280,3.901989,55.759704,37.642894,0,0
1,1,0,0,0,29,71,Sweet Glass Cookery,7.844818,4.484803,3.969818,4.729702,5.466900,5.641477,4.147588,55.727624,37.643497,1,2
2,2,18,0,0,0,82,Elderflower and Chops Inn,7.652421,9.122568,10.189378,9.322802,9.918229,6.908881,7.271825,55.725574,37.600146,1,1
3,3,58,0,42,0,0,Twisted Arrow Bar and Grill,3.850365,10.190916,3.586667,5.256077,6.760864,5.960194,1.289751,55.758699,37.607596,0,0
4,4,8,0,0,0,92,Jumping Olive Eatery,5.984862,7.656179,5.526642,5.210892,5.520184,4.257404,3.515553,55.754803,37.609356,0,0
5,5,0,91,0,0,9,Taste Bar and Grill,7.103353,5.452332,5.151574,10.445478,4.171451,10.009089,5.581476,55.759298,37.659743,0,2
6,6,14,0,86,0,0,Mad Chops Kitchen,1.577203,2.379689,2.901804,3.557828,4.862620,2.627251,2.109388,55.725476,37.620905,0,0
7,7,0,74,0,26,0,Splendid on Roadhouse,16.430692,6.589084,12.779909,8.402535,9.045163,11.105169,13.497808,55.778199,37.693352,2,2
8,8,0,51,0,49,0,Chef's Lion Kitchen,9.719800,8.797263,1.908809,4.007174,5.658806,8.225737,7.145497,55.772187,37.599329,2,1
9,9,84,0,16,0,0,Vista Lounge,7.132836,3.760264,3.384493,2.493711,3.504490,6.988876,7.289446,55.722860,37.676957,0,0


In [27]:
food = list(fp)[1:6]
wdays = list(fp)[7:14]

In [79]:
ldist = 3. # this parameter could be regulated due to transport
def request_food(lat, lng, estimated_food, ignore_days=list(), vegan=0):
    ans = dict()
    for wday in wdays:
        if wday in ignore_days: continue
        dist = fp[["latitude", "longitude"]].apply(lambda p: geodesic((lat, lng), p).km, axis=1)
        ww = fp[wday].to_numpy()
        func = ldist*dist.to_numpy()-ww
        A = (fp[food].to_numpy().transpose()/100)*np.tile(ww, (len(food), 1))
        b = np.array(list(map(estimated_food.get, food)))
        ans[wday] = spopt.linprog(func, A, b)
    return ans

In [80]:
request_food(55.762910, 37.619060, estimated_food={"Bakery":0, "Garniers":3, "Main":0, "Salads":1, "Soups":0})

{'Monday':      con: array([], dtype=float64)
      fun: -1.0250577520187807
  message: 'Optimization terminated successfully.'
      nit: 7
    slack: array([-3.03393726e-12,  1.95918367e+00, -2.55837002e-12, -5.25757216e-12,
        -5.54520931e-12])
   status: 0
  success: True
        x: array([2.33277377e-13, 1.01659032e-13, 1.00644371e-13, 5.32270772e-13,
        3.45015701e-13, 8.34350543e-13, 1.09027317e-13, 2.53076964e-13,
        2.09964856e-01, 8.20780204e-14, 4.98401770e-13, 1.65831333e-13,
        1.47124000e-13, 2.26149700e-13, 9.89163516e-14, 1.79800944e-12,
        9.05893460e-13]), 'Tuesday':      con: array([], dtype=float64)
      fun: -0.9185388590348275
  message: 'Optimization terminated successfully.'
      nit: 6
    slack: array([-7.90624060e-12,  1.95918367e+00, -7.22487418e-12, -4.64894789e-12,
        -1.25510797e-11])
   status: 0
  success: True
        x: array([5.67416522e-13, 2.14272797e-13, 3.98052774e-13, 7.65441307e-13,
        3.37785502e-13, 2.1944